In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
from peft import VeraConfig, get_peft_model, PeftModel
import numpy as np
import torch
import evaluate

In [8]:
torch.set_printoptions(threshold=torch.inf)  # Display all elements
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()

1

In [9]:
BASE_ID = "roberta-base"
tok  = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

base_model = AutoModelForSequenceClassification.from_pretrained(
        BASE_ID,
        num_labels=2,
        load_in_4bit=True,
        device_map="auto")

vera_cfg = VeraConfig(
        r=128,
        target_modules=["classifier.dense"],
        fan_in_fan_out=False,)


model = get_peft_model(base_model, vera_cfg)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(model)

PeftModel(
  (base_model): VeraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear4bit(in_features=768, out_features=768, bias=True)
                  (key): Linear4bit(in_features=768, out_features=768, bias=True)
                  (value): Linear4bit(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
             

In [11]:
stop

NameError: name 'stop' is not defined

In [ ]:
# q_proj = query
q_proj = model.base_model.model.roberta.encoder.layer[0].attention.self.query
print("q_proj trainable parameters:")
print(f"Σ shape: {q_proj.uilinlora_adapter['default'].shape}")
print(f"D shape: {getattr(q_proj, 'default_D').shape}")
print(f"E shape: {getattr(q_proj, 'default_E').shape}")
print(f"U shape: {getattr(q_proj, 'default_U').shape}")
print(f"V shape: {getattr(q_proj, 'default_V').shape}")

# v_proj = value
v_proj = model.base_model.model.roberta.encoder.layer[0].attention.self.value
print("\nv_proj trainable parameters:")
print(f"Σ shape: {v_proj.uilinlora_adapter['default'].shape}")
print(f"D shape: {getattr(v_proj, 'default_D').shape}")
print(f"E shape: {getattr(v_proj, 'default_E').shape}")
print(f"U shape: {getattr(v_proj, 'default_U').shape}")
print(f"V shape: {getattr(v_proj, 'default_V').shape}")


q_proj trainable parameters:


AttributeError: 'Linear4bit' object has no attribute 'default_D'

In [7]:
model.config.pad_token_id = tok.pad_token_id

# ---------- data ----------
raw_ds = load_dataset("glue", "sst2")

def tokenize(batch):
    natural   = tok(batch["sentence"], add_special_tokens=True)
    true_lens = [len(ids) for ids in natural["input_ids"]]

    padded = tok(
        batch["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    padded["real_length"] = true_lens
    return padded

tokenized_ds = raw_ds.map(
    tokenize,
    batched=True,
    remove_columns=["sentence", "idx"]
)

# rename + set Torch format
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels", "real_length"],
)

# ---------- stats ----------
max_len = max(tokenized_ds["train"]["real_length"])
print(f"Longest raw sentence: {max_len} tokens")


# ---------- data ----------
raw_datasets = load_dataset("glue", "sst2")
def tokenize_function(example):
    return tok(example["sentence"], truncation=True, padding="max_length", max_length=100)

# Tokenize the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
del raw_datasets
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Longest raw sentence: 74 tokens


In [8]:
# ---------- trainer ----------
args = TrainingArguments(
        output_dir="uilinlora-sst2",
        per_device_train_batch_size=32,
        num_train_epochs=1,
        learning_rate=3e-3,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=50)


trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"])

In [9]:
trainer.train()

/opt/anaconda3/envs/guyb_env2/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 GiB. GPU 0 has a total capacity of 23.59 GiB of which 12.30 GiB is free. Process 225277 has 9.97 GiB memory in use. Including non-PyTorch memory, this process has 1.29 GiB memory in use. Of the allocated memory 906.60 MiB is allocated by PyTorch, and 77.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
stop

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
logits = predictions.predictions[1]
preds = np.argmax(logits, axis=-1)
metric = evaluate.load("accuracy")

tokenized_datasets["validation"]["labels"]
metric.compute(predictions=preds, references=tokenized_datasets["validation"]["labels"])

In [ ]:
# after trainer.train()
adapter_dir = "diag_adapter"
model.save_pretrained(adapter_dir, safe_serialization=True)  # adapter only
tok.save_pretrained(adapter_dir)                             # optional, for easy reload

In [ ]:
base = AutoModelForSequenceClassification.from_pretrained(
           BASE_ID, load_in_4bit=True, device_map="auto")

model = PeftModel.from_pretrained(base, "uilinlora_adapter").to(device)

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
logits = predictions.predictions[1]
preds = np.argmax(logits, axis=-1)
metric = evaluate.load("accuracy")

tokenized_datasets["validation"]["labels"]
metric.compute(predictions=preds, references=tokenized_datasets["validation"]["labels"])

In [ ]:
# # # Debugging things

# # core_model   = model.get_base_model()        # → LlamaForSequenceClassification
# # llama_blocks = core_model.model.layers       # → ModuleList of decoder layers
# # qproj_0      = llama_blocks[0].self_attn.q_proj

# # print(type(qproj_0))          # should be your Linear4bit / Linear8bitLt
# # print(qproj_0.weight.shape)   # should be (out, in)  e.g.  (4096, 2048)

# model.train()
# with torch.amp.autocast("cuda"):
#     batch = tok(["hello"], return_tensors="pt").to(0)
#     out = model(**batch, labels=torch.tensor([1]).to(0))

# loss = out.loss.to(torch.float32)
# loss.backward()


# # Check grads manually
# for name, param in model.named_parameters():
#     if param.requires_grad and param.grad is not None:
#         print(f"{name} has non-zero grad: {param.grad.abs().mean().item():.6f}")

# model.train()
# batch = tok(["hello"], return_tensors="pt").to(0)
# out = model(**batch, labels=torch.tensor([1]).to(0))

# print("loss requires grad?", out.loss.requires_grad)
# print("loss grad_fn?", out.loss.grad_fn)

# for n, p in model.named_parameters():
#     if p.requires_grad:
#         print(n, p.shape)

#         # model.train()
# # batch = tok(["hello"], return_tensors="pt").to(0)
# # out = model(**batch, labels=torch.tensor([1]).to(0))
# # print(out.loss.grad_fn)  # should NOT be None



In [ ]:
# For non trained model accuracy 0.4919
# For r=128 one epoch lr 3e-3 accuracy 0.932
# For r=128 two epochs lr 3e-3 accuracy 0.939